In [37]:
!pip install -U langchain-community
! pip install chromadb

In [39]:
from langchain.document_loaders import WebBaseLoader

In [40]:
URL = ['https://peterbloem.nl/blog/transformers',
       'https://medium.com/@alejandro.itoaramendia/attention-is-all-you-need-a-complete-guide-to-transformers-8670a3f09d02' ,
       'https://medium.com/the-modern-scientist/detailed-explanations-of-transformer-step-by-step-dc32d90b3a98' ]


data = WebBaseLoader(URL)   #loading the data
content = data.load()       #extracting content


In [ ]:
content

In [41]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=256,chunk_overlap=50)
chunking = text_splitter.split_documents(content)

In [42]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

import os
from getpass import getpass

HF_token = 'Provide hugging face tokenn'
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_token

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key = HF_token,model_name = "BAAI/bge-base-en-v1.5"
)


In [43]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(chunking,embeddings) #storing embeddings in vector database

In [45]:
retriever = vectorstore.as_retriever(search_type="mmr",search_kwargs={"k":3})
query = "What is Transformer?"
docs_rel = retriever.get_relevant_documents(query)
print(docs_rel)

[Document(metadata={'description': 'The transformer is a model that follows an encoder-decoder architecture. Using attention, they capture complex patterns that understand a global context.', 'language': 'en', 'source': 'https://medium.com/@alejandro.itoaramendia/attention-is-all-you-need-a-complete-guide-to-transformers-8670a3f09d02', 'title': 'Attention : A Complete Guide to Transformers | Medium'}, page_content='within machine learning.The transformer is an architecture that relies on the concept of attention, a technique used to provide weights to different parts of an input sequence so that a better understanding of its underlying context is achieved. This'), Document(metadata={'description': 'Prior to the advent of the Transformer, the bag of words method is a commonly used approach in Natural Language Processing (NLP), treating each word or token as an independent entity in the context…', 'language': 'en', 'source': 'https://medium.com/the-modern-scientist/detailed-explanations-

In [46]:
prompt = f"""
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in context
</s>
<|user|>
{query}
</s>
<|assistant|>
"""                                                                                                                 #system prompt

In [48]:
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA

model = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha",
                       model_kwargs={"temperature":0.5,
                                     "max_new_tokens":512,
                                     "max_length":64
                                    })

qa = RetrievalQA.from_chain_type(llm=model,retriever=retriever,chain_type="stuff")
response = qa(prompt)
print(response['result'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

post “A Deep Look at Transformer Based Models”, I present detailed explanations of Google’s BERT, OpenAI’s GPT and other Transformer-based models.Also check out my YouTube Video Transformer based BERT and GPT, relevant to today’s post (in Chinese though

Building transformers
A transformer is not just a self-attention layer, it is an architecture. It’s not quite clear what does and doesn’t qualify as a transformer, but here we’ll use the following definition:

Attention : A Complete Guide to Transformers | MediumOpen in appSign upSign inWriteSign upSign inHomeLibraryStoriesStatsAttention Is All You Need : A Complete Guide to TransformersAlejandro Ito Aramendia·Follow11 min read·Jan 1, 2024--7ListenSharePhoto by

Question: 
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and gi